In [ ]:
from open_flamingo import create_model_and_transforms
import torch
from PIL import Image
from PIL import ImageFilter 
import requests

In [2]:
class PATH:
    lm_path = "path for mpt-7b"
    lm_tokenizer_path = "path for mpt-7b"
    checkpoint_path = "path for openflamingo v2 checkpoint.pt"
args = PATH()
device_set = 'cuda:0'
device = torch.device(device_set)

In [ ]:
flamingo,image_processor,tokenizer = create_model_and_transforms(
    clip_vision_encoder_path = 'ViT-L-14',
    clip_vision_encoder_pretrained = "openai",
    lang_encoder_path = args.lm_path,
    tokenizer_path = args.lm_tokenizer_path,
    cross_attn_every_n_layers=4,
    # new params
    inference=True,
    precision ='fp16',
    device = device_set,
    checkpoint_path = args.checkpoint_path,
)


Step 1: Setting and Loading images


In [123]:

demo_image_one = Image.open(
    requests.get(
        "http://images.cocodataset.org/val2017/000000039769.jpg", stream=True
    ).raw
)
# demo_image_one = Image.open("test-006.jpg")
# demo_image_two = Image.open(
#     requests.get(
#         "http://images.cocodataset.org/test-stuff2017/000000028137.jpg",
#         stream=True
#     ).raw
# )
demo_image_two = Image.open("test-006.jpg")
query_image = Image.open("test-006.jpg")

# query_image=query_image.filter(ImageFilter.GaussianBlur(radius=11))  
tokenizer.padding_side = "left" # For generation padding tokens should be on the left
# lang_x = tokenizer(
#     ["<image>Question:How many is the cats in the image? Short answer:two.<|endofchunk|><image>Question:What color is the wall? Short answer:white.<|endofchunk|><image>Question:How many is the oranges in the image? Short answer:"],
#     return_tensors="pt",
# )
# add the demonstrations question and answer.
lang_x = tokenizer(
    ["<image>Question: What kind of animals in the image? Answer: Dog. <|endofchunk|><image>Question: What kind of animals in the image? Answer: Dog. <|endofchunk|><image>Question: What kind of animals in the image? Answer:"],
    return_tensors="pt",
)

Step 2: Preprocessing and Generating Text

In [ ]:
vision_x = [image_processor(demo_image_one).unsqueeze(0), image_processor(demo_image_two).unsqueeze(0), image_processor(query_image).unsqueeze(0)]
vision_x = torch.cat(vision_x, dim=0)
vision_x = vision_x.unsqueeze(1).unsqueeze(0)
# load data to gpus
vision_x = vision_x.to(device).half()
print(vision_x.device)
input_ids=lang_x["input_ids"]
attention_mask = lang_x["attention_mask"]
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)

generated_text = flamingo.generate(
    vision_x=vision_x,
    lang_x=input_ids,
    attention_mask=attention_mask,
    max_new_tokens=20,
    num_beams=3,
)
print(tokenizer.decode(generated_text[0]))